In [1]:
import pandas as pd
import re

In [2]:
tesla = pd.read_csv('../../data/full_tesla_df.com.csv')
tesla.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
tesla

,sell_type,car_type,year,color,country,traction,price,km,range,max_vel,zero_hundred
0,Nuevo,Model S,2024,WHITE,spain,"Model S, tracción a las cuatro ruedas",99.600 €,Cuentakilómetros con 503 km,634.0,250.0,"3,2"
1,Nuevo,Model S,2024,WHITE,spain,"Model S, tracción a las cuatro ruedas",102.270 €,Cuentakilómetros con 99 km,634.0,250.0,"3,2"
2,Nuevo,Model S,2024,WHITE,spain,"Model S, tracción a las cuatro ruedas",102.270 €,0,634.0,250.0,"3,2"
3,Nuevo,Model S,2024,BLACK,spain,"Model S, tracción a las cuatro ruedas",90.340 €,Cuentakilómetros con 577 km,634.0,250.0,"3,2"
4,Nuevo,Model S,2024,BLACK,spain,Plaid,101.050 €,Cuentakilómetros con 3652 km,600.0,322.0,"2,1"
...,...,...,...,...,...,...,...,...,...,...,...
2058,Ocasion,Model Y,2022,WHITE,italy,Performance a trazione integrale,45.500 €,69.439 km sul contachilometri,514.0,250.0,"3,7"
2059,Ocasion,Model Y,2023,BLACK,italy,Long Range a trazione integrale,42.000 €,82.410 km sul contachilometri,533.0,217.0,5
2060,Ocasion,Model Y,2022,BLACK,italy,Performance a trazione integrale,43.000 €,92.596 km sul contachilometri,514.0,250.0,"3,7"
2061,Ocasion,Model Y,2022,GREY,italy,Long Range a trazione integrale,41.000 €,48.960 km sul contachilometri,533.0,217.0,5


In [3]:
tesla['traction'].unique()

array(['Model S, tracción a las cuatro ruedas', 'Plaid',
       'Model 3 Tracción trasera', 'Gran autonomía con tracción integral',
       'Model 3 Gran autonomía con tracción trasera',
       'Performance con tracción a las cuatro ruedas',
       'Model X, tracción a las cuatro ruedas',
       'Model Y Gran autonomía con tracción trasera',
       'Model Y Tracción trasera', '75D', '100D Gran autonomía',
       'Model S com tração integral',
       'Model 3, Long Range, tração traseira',
       'Long Range, tração integral', nan, 'Model 3, tração traseira',
       'Performance tração integral', 'Model X com tração integral',
       'Model Y, tração traseira', 'Model Y, Long Range, tração traseira',
       '100D Ludicrous Performance', 'Model S Transmission intégrale',
       'Model 3 Propulsion', 'Model 3 Grande Autonomie, Propulsion',
       'Grande Autonomie, transmission intégrale',
       'Performance Transmission Intégrale',
       'Model X Transmission intégrale', 'Model Y Propul

In [4]:
# Crear una función para evaluar patrones
def contains_pattern(text, patterns):
    if pd.isna(text):
        return False
    for pattern in patterns:
        if pattern in text:
            return True
    return False

In [5]:
# Definir patrones para cada categoría
high_range_patterns = [
    'Gran autonomía', 'Grande Autonomie', 'Long Range', '100D', 'Autonomie Standard Plus'
]
performance_patterns = [
    'Plaid', 'Performance', 'Ludicrous'
]
four_wheel_drive_patterns = [
    'tracción a las cuatro ruedas', 'tracción integral', 'com tração integral',
    'transmission intégrale', 'a trazione integrale'
]
rear_wheel_drive_patterns = [
    'tracción trasera', 'tração traseira', 'Propulsion', 'a trazione posteriore'
]

In [6]:
# Crear columnas
tesla['high_range'] = tesla['traction'].apply(lambda x: contains_pattern(x, high_range_patterns))
tesla['performance'] = tesla['traction'].apply(lambda x: contains_pattern(x, performance_patterns))
tesla['four_wheel_drive'] = tesla['traction'].apply(lambda x: contains_pattern(x, four_wheel_drive_patterns))
tesla['rear_wheel_drive'] = tesla['traction'].apply(lambda x: contains_pattern(x, rear_wheel_drive_patterns))

tesla.drop(columns=['traction'], inplace=True)

In [7]:
# Dropear filas con valores nulos en car_type y price
tesla.dropna(subset=["car_type", "price"], inplace=True)

# Limpiar y convertir la columna 'km'
tesla['km'] = tesla['km'].str.extract(r'(\d+)').astype(float)

tesla['zero_hundred'] = (
    tesla['zero_hundred']
    .str.replace(',', '.', regex=False)
    .astype(float)
)

# Limpiar y convertir la columna 'price'
tesla['price'] = (
    tesla['price']
    .str.replace('€', '', regex=False)
    .str.replace(r'[^\d]', '', regex=True)
    .astype(float)
)

tesla

,sell_type,car_type,year,color,country,price,km,range,max_vel,zero_hundred,high_range,performance,four_wheel_drive,rear_wheel_drive
0,Nuevo,Model S,2024,WHITE,spain,99600.0,503.0,634.0,250.0,3.2,False,False,True,False
1,Nuevo,Model S,2024,WHITE,spain,102270.0,99.0,634.0,250.0,3.2,False,False,True,False
2,Nuevo,Model S,2024,WHITE,spain,102270.0,0.0,634.0,250.0,3.2,False,False,True,False
3,Nuevo,Model S,2024,BLACK,spain,90340.0,577.0,634.0,250.0,3.2,False,False,True,False
4,Nuevo,Model S,2024,BLACK,spain,101050.0,3652.0,600.0,322.0,2.1,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,Ocasion,Model Y,2022,WHITE,italy,45500.0,69.0,514.0,250.0,3.7,False,True,True,False
2059,Ocasion,Model Y,2023,BLACK,italy,42000.0,82.0,533.0,217.0,5.0,True,False,True,False
2060,Ocasion,Model Y,2022,BLACK,italy,43000.0,92.0,514.0,250.0,3.7,False,True,True,False
2061,Ocasion,Model Y,2022,GREY,italy,41000.0,48.0,533.0,217.0,5.0,True,False,True,False


In [8]:
tesla = pd.get_dummies(data=tesla, columns=['sell_type', 'car_type', 'color', 'country'])
tesla

,year,price,km,range,max_vel,zero_hundred,high_range,performance,four_wheel_drive,rear_wheel_drive,...,color_BLUE,color_CHERRY,color_GREY,color_RED,color_SILVER,color_WHITE,country_france,country_italy,country_portugal,country_spain
0,2024,99600.0,503.0,634.0,250.0,3.2,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
1,2024,102270.0,99.0,634.0,250.0,3.2,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
2,2024,102270.0,0.0,634.0,250.0,3.2,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
3,2024,90340.0,577.0,634.0,250.0,3.2,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
4,2024,101050.0,3652.0,600.0,322.0,2.1,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,2022,45500.0,69.0,514.0,250.0,3.7,False,True,True,False,...,False,False,False,False,False,True,False,True,False,False
2059,2023,42000.0,82.0,533.0,217.0,5.0,True,False,True,False,...,False,False,False,False,False,False,False,True,False,False
2060,2022,43000.0,92.0,514.0,250.0,3.7,False,True,True,False,...,False,False,False,False,False,False,False,True,False,False
2061,2022,41000.0,48.0,533.0,217.0,5.0,True,False,True,False,...,False,False,True,False,False,False,False,True,False,False


In [9]:
tesla.drop_duplicates(inplace=True)
tesla

,year,price,km,range,max_vel,zero_hundred,high_range,performance,four_wheel_drive,rear_wheel_drive,...,color_BLUE,color_CHERRY,color_GREY,color_RED,color_SILVER,color_WHITE,country_france,country_italy,country_portugal,country_spain
0,2024,99600.0,503.0,634.0,250.0,3.2,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
1,2024,102270.0,99.0,634.0,250.0,3.2,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
2,2024,102270.0,0.0,634.0,250.0,3.2,False,False,True,False,...,False,False,False,False,False,True,False,False,False,True
3,2024,90340.0,577.0,634.0,250.0,3.2,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
4,2024,101050.0,3652.0,600.0,322.0,2.1,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,2022,45500.0,69.0,514.0,250.0,3.7,False,True,True,False,...,False,False,False,False,False,True,False,True,False,False
2059,2023,42000.0,82.0,533.0,217.0,5.0,True,False,True,False,...,False,False,False,False,False,False,False,True,False,False
2060,2022,43000.0,92.0,514.0,250.0,3.7,False,True,True,False,...,False,False,False,False,False,False,False,True,False,False
2061,2022,41000.0,48.0,533.0,217.0,5.0,True,False,True,False,...,False,False,True,False,False,False,False,True,False,False


In [10]:
tesla.to_csv('../../data/full_cleaned_data.csv')